## Import Python libraries

In [46]:
!pip install flwr

In [47]:
import tensorflow.compat.v1 as tf
sess = tf.Session()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))
import errno
import tensorflow as tf
import csv
from keras import backend as K
import sklearn
from tensorflow import keras
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPool1D
from keras.optimizers import Adam
import random
import pandas as pd
import numpy as np
import math
import os
import flwr as fl
import array
from scipy.stats import dirichlet
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE

['adult-census-income']


In [48]:
!pip install aif360 

In [49]:
!pip install BlackBoxAuditing

## Import dataset

In [50]:
path = ('/kaggle/input/adult-census-income/adult.csv')
df = pd.read_csv(path, encoding='latin-1')

## Exploratory Data Analysis

### Check shape of dataset

In [51]:
df.shape

(32561, 15)

We can see that there are 32561 instances and 15 attributes in the data set.

### Preview dataset

In [52]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [53]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

### View summary of dataframe

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Summary of the dataset shows that there are no missing values. But the preview shows that the dataset contains values coded as `?`. So, I will encode `?` as NaN values.

### Encode `?` as `NaNs`

In [55]:
df[df == '?'] = np.nan

In [56]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [57]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

### Impute missing values with mode

In [58]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)

### Check again for missing values

In [59]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [60]:
def Dirichelet_sampling(data, alphas, values, n_lines):
    """
        Produit une dataset suivant la distribution de Dirichlet paramétrée par le vecteur alphas
       sur les valeurs values
        e.g. values = [Male, Female] --> Le nombre d'individus correspondant à Male et Female va suivre la
        distrib Dir(alphas)
    """
    # values = data[col].unique()
    assert (n_lines <= len(data.axes[0]))
    assert (len(values) == len(alphas))
    print(len(data.axes[0]))
    # sample a distribution from dir(alphas)
    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    print(s)
    print(alphas)
    groups = []
    for i in range(len(values)):
        if values[0] == 'Male' or values[0] == 'Female':
            if round(n_lines * s[0]) > 0:
                groups.append(data[data['sex'] == 1.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

            if round(n_lines * s[1]) > 0:
                groups.append(data[data['sex'] == 0.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

        else:
            if round(n_lines * s[i]) > 0:
                groups.append(data[data[values[i]] == 1.0].sample(n=round(n_lines * s[i]), replace=True))
            else:
                groups.append(data[data[values[i]] == 1.0].sample(n=1))

    return pd.concat(groups)

In [61]:
df.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

Now we can see that there are no missing values in the dataset.

### Setting feature vector and target variable

In [62]:
X = df.drop(['income'], axis=1)
y = df['income']

In [63]:
y.unique(),y.nunique()

(array(['<=50K', '>50K'], dtype=object), 2)

In [64]:
X.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,Private,186061,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


## Split data into separate training and test set

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Feature Engineering

### Encode categorical variables: LabelEncoder

![](https://ai-ml-analytics.com/wp-content/uploads/2020/08/encoding-3.png)

In [66]:
from sklearn import preprocessing

categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [67]:
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
dff = pd.concat([X, y], axis=1)
dff=dff.reset_index(drop=True)

In [68]:
def Dirichelet_sampling(data, alphas, values, n_lines):
    """
        Produit une dataset suivant la distribution de Dirichlet paramétrée par le vecteur alphas
       sur les valeurs values
        e.g. values = [Male, Female] --> Le nombre d'individus correspondant à Male et Female va suivre la
        distrib Dir(alphas)
    """
    # values = data[col].unique()
    assert (n_lines <= len(data.axes[0]))
    assert (len(values) == len(alphas))
    print(len(data.axes[0]))
    # sample a distribution from dir(alphas)
    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    print(s)
    print(alphas)
    groups = []
    for i in range(len(values)):
        if values[0] == 'Male' or values[0] == 'Female':
            if round(n_lines * s[0]) > 0:
                groups.append(data[data['sex'] == 1.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

            if round(n_lines * s[1]) > 0:
                groups.append(data[data['sex'] == 0.0].sample(n=round(n_lines * s[0]), replace=True))
            else:
                groups.append(data[data['sex'] == 1.0].sample(n=1))

        else:
            if round(n_lines * s[i]) > 0:
                groups.append(data[data[values[i]] == 1.0].sample(n=round(n_lines * s[i]), replace=True))
            else:
                groups.append(data[data[values[i]] == 1.0].sample(n=1))

    return pd.concat(groups)

In [69]:
values = ["Male", "Female"]
df1 = Dirichelet_sampling(dff, [0.2,0.8], values, 11000)
df2 = Dirichelet_sampling(dff, [0.7,0.9], values, 10000)
df3 = Dirichelet_sampling(dff, [0.6,0.3], values, 10000)

32561
[2.2615133871811283e-05, 0.9999773848661283]
[0.2, 0.8]
32561
[0.9948943626563729, 0.005105637343627288]
[0.7, 0.9]
32561
[0.9995171829167762, 0.0004828170832238118]
[0.6, 0.3]


In [70]:
df1.shape,df2.shape, df3.shape

((2, 15), (39796, 15), (39980, 15))

In [71]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

## Feature Scaling

![](https://cdn-images-1.medium.com/max/370/1*Nlgc_wq2b-VfdawWX9MLWA.png)

In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [73]:
X_train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,0.101484,2.600478,-1.494279,-0.332263,1.133894,-0.402341,-0.782234,2.214196,0.39298,-1.430470,-0.145189,-0.217407,-1.662414,0.262317
1,0.028248,-1.884720,0.438778,0.184396,-0.423425,-0.402341,-0.026696,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.200753,0.262317
2,0.247956,-0.090641,0.045292,1.217715,-0.034095,0.926666,-0.782234,-0.276689,0.39298,-1.430470,-0.145189,-0.217407,-0.038346,0.262317
3,-0.850587,-1.884720,0.793152,0.184396,-0.423425,0.926666,-0.530388,0.968753,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317
4,-0.044989,-2.781760,-0.853275,0.442726,1.523223,-0.402341,-0.782234,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317


## Modeling + Fairness

**Remove the 'race' and 'sex' columns from the dataset to remove any potential biases or discriminatory factors**

In [74]:
X_trainn = X_train.drop(['race', 'sex'], axis=1)
X_testt = X_test.drop(['race', 'sex'], axis=1)

In [75]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
model = DecisionTreeRegressor()
model.fit(X_trainn, y_train)

y_pred = model.predict(X_testt)

mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.18947691677756168


****

# adversarial debiasing :
is an in-processing technique for fairness in machine learning that tries to learn a fair classifier by adding an adversary to the training process.

The goal is to train a model that is both accurate and fair with respect to some protected attributes, such as race or gender. Adversarial debiasing adds a new step to the training process, where an adversarial network is trained to predict the protected attribute from the hidden layers of the main network. This adversarial network tries to maximize the error in predicting the protected attribute while the main network tries to minimize it.

By doing so, the main network learns to encode the input features in a way that is less informative about the protected attributes, which leads to a fairer classifier.

In [76]:
!pip install aif360

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing

In [77]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

y_train_df = pd.DataFrame({'income': y_train})

dataset_train = StandardDataset(
    df=X_train.join(y_train_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)

In [78]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
debias_model = AdversarialDebiasing(
    unprivileged_groups=[{'race': 0, 'sex': 0}],
    privileged_groups=[{'race': 1, 'sex': 1}],
    scope_name='adversarial_debiasing',
    sess=sess
)
debias_model.fit(dataset_train)

epoch 0; iter: 0; batch classifier loss: 0.753187; batch adversarial loss: 1.090124
epoch 1; iter: 0; batch classifier loss: 0.427492; batch adversarial loss: 0.861694
epoch 2; iter: 0; batch classifier loss: 0.336720; batch adversarial loss: 0.579070
epoch 3; iter: 0; batch classifier loss: 0.302160; batch adversarial loss: 0.495438
epoch 4; iter: 0; batch classifier loss: 0.302930; batch adversarial loss: 0.423952
epoch 5; iter: 0; batch classifier loss: 0.294982; batch adversarial loss: 0.271953
epoch 6; iter: 0; batch classifier loss: 0.346463; batch adversarial loss: 0.279498
epoch 7; iter: 0; batch classifier loss: 0.316812; batch adversarial loss: 0.134161
epoch 8; iter: 0; batch classifier loss: 0.356158; batch adversarial loss: 0.310496
epoch 9; iter: 0; batch classifier loss: 0.232283; batch adversarial loss: 0.126941
epoch 10; iter: 0; batch classifier loss: 0.329774; batch adversarial loss: 0.394236
epoch 11; iter: 0; batch classifier loss: 0.260891; batch adversarial loss:

In [79]:
y_test_df = pd.DataFrame({'income': y_test})
dataset_test = StandardDataset(
    df=X_test.join(y_test_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)
dataset_debiased = debias_model.predict(dataset_test)

In [80]:
# Evaluate the performance of the debiased model
y_pred = dataset_debiased.labels
y_true = dataset_test.labels.ravel()
print("Accuracy score:", accuracy_score(y_true, y_pred))
print("Balanced accuracy score:", balanced_accuracy_score(y_true, y_pred))
print("F1 score:", f1_score(y_true, y_pred))

Accuracy score: 0.8463507011976661
Balanced accuracy score: 0.7487499563791926
F1 score: 0.6377021482017861


Accuracy score: measures the proportion of correctly classified instances among all instances. It is computed as the number of true positives plus true negatives divided by the total number of instances. It is not suitable for imbalanced datasets, where the classes have very different sizes.

Balanced accuracy score: a modified version of accuracy score that takes into account the class imbalance by computing the average of the sensitivity (true positive rate) and specificity (true negative rate). It is computed as (sensitivity + specificity) / 2.

F1 score: measures the trade-off between precision and recall. It is the harmonic mean of precision and recall, computed as 2 * (precision * recall) / (precision + recall). It is a good metric for imbalanced datasets, where the focus is on correctly identifying the minority class.

****

# Disparate Impact Remover
is a pre-processing algorithm used to mitigate the effects of sensitive attributes (such as race or gender) on the outcomes of machine learning models. The algorithm works by adjusting the distribution of the sensitive attribute in the dataset to achieve a desired level of fairness.

The Disparate Impact Remover algorithm works by creating a new dataset in which the sensitive attribute (such as race or gender) is balanced across the different values of the outcome variable. This is achieved by adjusting the probability of selecting each example in the training dataset, such that the overall proportion of the sensitive attribute is the same across all possible outcomes. This approach is based on the intuition that if the dataset is balanced with respect to the sensitive attribute, then the machine learning model will not be able to discriminate based on this attribute.

In [81]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [82]:
train_dataset = BinaryLabelDataset(df=X_train.join(y_train_df), label_names=['income'], 
                                   protected_attribute_names=['race', 'sex'], 
                                   favorable_label=1, unfavorable_label=0)
test_dataset = BinaryLabelDataset(df=X_test.join(y_test_df), label_names=['income'], 
                                  protected_attribute_names=['race', 'sex'], 
                                  favorable_label=1, unfavorable_label=0)

In [83]:
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute='race')
train_repaired_dataset = DIR.fit_transform(train_dataset)

In [84]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_repaired_dataset.features, train_repaired_dataset.labels.ravel())

RandomForestClassifier()

In [85]:
y_pred = rfc.predict(test_dataset.features)

In [86]:
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.8467601596888116
Precision: 0.7268421052631578
Recall: 0.5854175498092412
F1 score: 0.6485090396806762


****

# Learning fair representations (LFR) 
is a technique for learning representations of data that are optimized to remove or reduce the impact of sensitive attributes such as race or gender. It is a type of pre-processing technique that can be applied to the data before it is used to train a machine learning model.

The goal of LFR is to find a representation of the data that separates the information that is relevant to the prediction task from the information that is associated with the sensitive attribute. This representation can then be used as input to a machine learning model that can make predictions without relying on the sensitive attribute.

In [87]:
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [88]:
train_dataset  = StandardDataset(
    df=X_train.join(y_train_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']])
test_dataset  = StandardDataset(
    df=X_test.join(y_test_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']])


In [89]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = rfc.predict(test_dataset.features)
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


Accuracy: 0.8543351417750026
Precision: 0.7452830188679245
Recall: 0.602797795676134
F1 score: 0.6665104288727444


Reweighing

In [90]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

# Création des jeux de données d'entraînement et de test avec AIF360
train_dataset = BinaryLabelDataset(df=X_train.join(y_train_df), label_names=['income'], 
                                   protected_attribute_names=['race', 'sex'], 
                                   favorable_label=1, unfavorable_label=0)

test_dataset = BinaryLabelDataset(df=X_test.join(y_test_df), label_names=['income'], 
                                  protected_attribute_names=['race', 'sex'], 
                                  favorable_label=1, unfavorable_label=0)

# Définition des groupes privilégiés et non privilégiés
privileged_groups = [{'race': 1}]  # Remplacez par vos propres valeurs
unprivileged_groups = [{'race': 0}]  # Remplacez par vos propres valeurs

# Application de la méthode de reweighing
rw = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
train_dataset_reweighed = rw.fit_transform(train_dataset)

# Entraînement d'un modèle de RandomForestClassifier sur les données reweighed
rfc = RandomForestClassifier()
rfc.fit(train_dataset_reweighed.features, train_dataset_reweighed.labels.ravel())

# Prédictions sur l'ensemble de test
X_test_reweighed = rw.transform(test_dataset)
y_pred = rfc.predict(X_test_reweighed.features)

# Calcul des mesures de performance
accuracy = accuracy_score(test_dataset.labels.ravel(), y_pred)
precision = precision_score(test_dataset.labels.ravel(), y_pred)
recall = recall_score(test_dataset.labels.ravel(), y_pred)
f1 = f1_score(test_dataset.labels.ravel(), y_pred)

print("Exactitude:", accuracy)
print("Précision:", precision)
print("Rappel:", recall)
print("Score F1:", f1)


Exactitude: 0.8577131743269526
Précision: 0.7501290655653072
Rappel: 0.6159389571852479
Score F1: 0.6764432029795159


In [101]:
type(train_dataset_reweighed)

aif360.datasets.binary_label_dataset.BinaryLabelDataset

In [112]:
train_dataset_reweighed.labels.ravel()

array([0., 1., 0., ..., 0., 1., 1.])

In [108]:
train_dataset.features.shape

(22792, 14)

In [104]:
train_dataset_reweighed.features.shape

(22792, 14)